[View in Colaboratory](https://colab.research.google.com/github/kyokicchi/ggl_an_codes/blob/master/trainLGBmodel.ipynb)

In [1]:
!pip install lightgbm

    100% |████████████████████████████████| 1.1MB 11.9MB/s 


In [0]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from google.colab import drive
from google.colab import files
from sklearn.model_selection import train_test_split
import pickle

In [3]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
%cd ../content/gdrive/My Drive/projects/ggl_an/
%ls

/content/gdrive/My Drive/projects/ggl_an
model_LGB.sav  model_XGB.sav  user_test_n.csv
model_NN.h5    output.csv     user_train_n.csv


In [0]:
s_data = 'user_train_n.csv'
filename = 'model_LGB.sav'
s_col_tgt = 'transactionRevenue_sum'
l_col_drop = [s_col_tgt]

In [6]:
%%time
df_train = pd.read_csv(s_data, index_col = 0)

<string>:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 8.34 s, sys: 1.04 s, total: 9.37 s
Wall time: 12.4 s


In [7]:
%%time
df_y = df_train[s_col_tgt]
df_x = df_train.drop(l_col_drop, axis=1)

CPU times: user 65 ms, sys: 186 ms, total: 251 ms
Wall time: 250 ms


In [9]:
%%time
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y)
x_eval, x_valid, y_eval, y_valid = train_test_split(x_test, y_test)



CPU times: user 944 ms, sys: 139 ms, total: 1.08 s
Wall time: 1.09 s


In [0]:

params = {
    'learning_rate': 0.03,
    'n_estimators': 200,
    'num_leaves': 128,
    'subsample': 0.2217,
    'colsample_bytree': 0.6810,
    'min_split_gain': np.power(10.0, -4.9380),
    'reg_alpha': np.power(10.0, -3.2454),
    'reg_lambda': np.power(10.0, -4.8571),
    'min_child_weight': np.power(10.0, 2),
    'silent': True
}

model = lgb.LGBMRegressor(**params)



In [11]:
%%time
model.fit(x_train, 
          y_train,
          eval_set=[(x_train, y_train),(x_eval, y_eval)],
          eval_metric='rmse',
          early_stopping_rounds=100)

[1]	training's rmse: 2.90526	training's l2: 8.44056	valid_1's rmse: 2.79103	valid_1's l2: 7.78984
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 2.87437	training's l2: 8.26202	valid_1's rmse: 2.76125	valid_1's l2: 7.6245
[3]	training's rmse: 2.84412	training's l2: 8.08902	valid_1's rmse: 2.73133	valid_1's l2: 7.46014
[4]	training's rmse: 2.81569	training's l2: 7.92809	valid_1's rmse: 2.7039	valid_1's l2: 7.31108
[5]	training's rmse: 2.78798	training's l2: 7.77284	valid_1's rmse: 2.6771	valid_1's l2: 7.16688
[6]	training's rmse: 2.76148	training's l2: 7.6258	valid_1's rmse: 2.65108	valid_1's l2: 7.0282
[7]	training's rmse: 2.73589	training's l2: 7.48507	valid_1's rmse: 2.62596	valid_1's l2: 6.89565
[8]	training's rmse: 2.71249	training's l2: 7.35763	valid_1's rmse: 2.60349	valid_1's l2: 6.77818
[9]	training's rmse: 2.68897	training's l2: 7.23054	valid_1's rmse: 2.58064	valid_1's l2: 6.65968
[10]	training's rmse: 2.66658	training's l2: 7.11063	valid_1

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.681,
       importance_type='split', learning_rate=0.03, max_depth=-1,
       min_child_samples=20, min_child_weight=100.0,
       min_split_gain=1.153453257821093e-05, n_estimators=200, n_jobs=-1,
       num_leaves=128, objective=None, random_state=None,
       reg_alpha=0.0005683292391383383, reg_lambda=1.3896326196366016e-05,
       silent=True, subsample=0.2217, subsample_for_bin=200000,
       subsample_freq=0)

In [12]:
%%time
pickle.dump(model, open(filename, 'wb'))

# files.download(filename) 

CPU times: user 233 ms, sys: 3.07 ms, total: 236 ms
Wall time: 933 ms
